In [16]:
import numpy as np
import cv2 as cv
import glob

In [17]:
def prepare_object_points(rows, cols):
    """Prepare object points for chessboard corners."""
    objp = np.zeros((rows * cols, 3), np.float32)
    objp[:, :2] = np.mgrid[0:cols, 0:rows].T.reshape(-1, 2)
    return objp

In [18]:
def convert_to_grayscale_and_threshold(img_path):
    """Convert image to grayscale and apply thresholding."""
    img = cv.imread(img_path)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    _, gray = cv.threshold(gray, 127, 255, cv.THRESH_BINARY)
    return gray

def check_chessboard_and_find_corners(gray, rows, cols, criteria):
    """Check for chessboard corners and refine them."""
    if not cv.checkChessboard(gray, (cols, rows)):
        return False, None

    ret, corners = cv.findChessboardCornersSB(gray, (cols, rows), flags=0)
    if ret:
        corners = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
    return ret, corners

In [19]:
def process_images(image_paths, rows, cols, criteria):
    """Process each image to find chessboard corners."""
    objpoints = []
    imgpoints = []
    objp = prepare_object_points(rows, cols)

    for img_path in image_paths:
        gray = convert_to_grayscale_and_threshold(img_path)
        ret, corners = check_chessboard_and_find_corners(gray, rows, cols, criteria)

        if ret:
            objpoints.append(objp)
            imgpoints.append(corners)
            img = cv.imread(img_path)
            cv.drawChessboardCorners(img, (cols, rows), corners, ret)
            cv.imwrite(img_path.replace(".png", "_new.png").replace("Images", "Outputs"), img)
            cv.imshow('img', img)
            cv.waitKey(500)

    cv.destroyAllWindows()
    return objpoints, imgpoints, gray.shape[::-1]

def process_images_exclude(image_paths, rows, cols, criteria):
    """Process each image but exclude './Images/5.png'."""
    objpoints = []
    imgpoints = []
    objp = prepare_object_points(rows, cols)

    for img_path in image_paths:
        if img_path == './Images/5.png':
            print(f"Excluding {img_path} from processing.")
            continue
        
        gray = convert_to_grayscale_and_threshold(img_path)
        ret, corners = check_chessboard_and_find_corners(gray, rows, cols, criteria)

        if ret:
            objpoints.append(objp)
            imgpoints.append(corners)
            img = cv.imread(img_path)
            cv.drawChessboardCorners(img, (cols, rows), corners, ret)
            cv.imwrite(img_path.replace(".png", "_new.png").replace("Images", "Outputs"), img)
            cv.imshow('img', img)
            cv.waitKey(500)

    cv.destroyAllWindows()
    return objpoints, imgpoints, gray.shape[::-1]

In [20]:
def calibrate_camera(objpoints, imgpoints, image_shape):
    """Calibrate the camera using object and image points."""
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, image_shape, None, None)
    newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (image_shape[1], image_shape[0]), 1, (image_shape[1], image_shape[0]))
    return ret, mtx, dist, rvecs, tvecs, newcameramtx, roi

In [21]:
def print_rotation_matrices_tvecs(rvecs, tvecs, image_paths):
    """Print rotation matrices and translation vectors for each image."""
    for i, (rvec, tvec) in enumerate(zip(rvecs, tvecs)):
        rotation_matrix, _ = cv.Rodrigues(rvec)  # Convert rvec to rotation matrix
        print(f"Image: {image_paths[i]}")
        print(f"Rotation Matrix:\n{rotation_matrix}")
        print(f"Translation Vector (tvec):\n{tvec}")
        print("-------------------------------------------")

In [22]:
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
rows, cols = 7, 7
images = glob.glob('./Images/*.png')

# Process images excluding './Images/5.png'
objpoints_exclude, imgpoints_exclude, image_shape = process_images_exclude(images, rows, cols, criteria)

# Calibrate camera using points *excluding* './Images/5.png'
ret_exclude, mtx_exclude, dist_exclude, rvecs_exclude, tvecs_exclude, newcameramtx_exclude, roi_exclude = calibrate_camera(objpoints_exclude, imgpoints_exclude, image_shape)

# Process all images without excluding any
objpoints_include, imgpoints_include, _ = process_images(images, rows, cols, criteria)

# Calibrate camera using points *including* './Images/5.png'
ret_include, mtx_include, dist_include, rvecs_include, tvecs_include, newcameramtx_include, roi_include = calibrate_camera(objpoints_include, imgpoints_include, image_shape)

Excluding ./Images/5.png from processing.


In [23]:
print("Results including './Images/5.png':")
print(f"Reprojection error: {ret_include}")
print("Camera matrix:\n", mtx_include)
print("Distortion coefficients:\n", dist_include)
print("New camera matrix:\n", newcameramtx_include)
print("Region of interest (ROI):\n", roi_include)
print_rotation_matrices_tvecs(rvecs_include, tvecs_include, images)

Results including './Images/5.png':
Reprojection error: 0.9151344822909565
Camera matrix:
 [[440.44093701   0.         228.23975227]
 [  0.         439.21581056 261.18189485]
 [  0.           0.           1.        ]]
Distortion coefficients:
 [[ 9.96413382e-03  5.85713600e-01 -5.64285416e-03 -8.65693134e-04
  -1.30946286e+00]]
New camera matrix:
 [[437.18933105   0.         226.90641969]
 [  0.         331.27285767 265.6536181 ]
 [  0.           0.           1.        ]]
Region of interest (ROI):
 (6, 74, 567, 319)
Image: ./Images/3.png
Rotation Matrix:
[[ 0.99976092  0.01073925  0.01904676]
 [-0.01071719  0.99994178 -0.00125966]
 [-0.01905918  0.00105523  0.9998178 ]]
Translation Vector (tvec):
[[-3.64602831]
 [-1.95172863]
 [10.29052005]]
-------------------------------------------
Image: ./Images/5.png
Rotation Matrix:
[[ 0.74952068 -0.50754825 -0.4249865 ]
 [ 0.62856848  0.74704491  0.21639216]
 [ 0.20765454 -0.42932352  0.87895444]]
Translation Vector (tvec):
[[-0.65502528]
 [-2.

In [24]:
print("\nResults excluding './Images/5.png':")
print(f"Reprojection error: {ret_exclude}")
print("Camera matrix:\n", mtx_exclude)
print("Distortion coefficients:\n", dist_exclude)
print("New camera matrix:\n", newcameramtx_exclude)
print("Region of interest (ROI):\n", roi_exclude)
print_rotation_matrices_tvecs(rvecs_exclude, tvecs_exclude, [img for img in images if img != './Images/5.png'])


Results excluding './Images/5.png':
Reprojection error: 0.45580715377987446
Camera matrix:
 [[418.96935841   0.         215.82538587]
 [  0.         418.37508427 291.10248945]
 [  0.           0.           1.        ]]
Distortion coefficients:
 [[ 2.68437103e-03  7.22163183e-01 -2.84614988e-03 -1.97902047e-03
  -2.83940778e+00]]
New camera matrix:
 [[379.17211914   0.         249.13278597]
 [  0.         344.30960083 239.56824185]
 [  0.           0.           1.        ]]
Region of interest (ROI):
 (54, 0, 522, 356)
Image: ./Images/3.png
Rotation Matrix:
[[ 0.99990273  0.01004572  0.00967574]
 [-0.01011196  0.9999256   0.00682158]
 [-0.0096065  -0.00691875  0.99992992]]
Translation Vector (tvec):
[[-3.35459316]
 [-2.65087877]
 [ 9.78244261]]
-------------------------------------------
Image: ./Images/2.png
Rotation Matrix:
[[ 0.9996534  -0.00772618 -0.02516705]
 [ 0.02091923  0.81350251  0.58118504]
 [ 0.01598312 -0.58151008  0.81338218]]
Translation Vector (tvec):
[[-2.95833808]
 [-